In [1]:
import pandas as pd
import glob
from matplotlib import pyplot
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import warnings

from collections import Counter
from tqdm import tqdm
import os
import math

In [2]:
warnings.filterwarnings("ignore")

In [3]:
# function that finds the indexes of non-anomalies for interpolation 
def interpolation_indexes(mylist, mynumber):
    
    left_neighbour = 0
    right_neighbour = 0
    
    # check left neighbour
    if((mynumber - 1) not in mylist):
        left_neighbour = mynumber - 1
    else:
        min_number = mynumber
        while min_number in mylist:
            min_number = min_number - 1
        left_neighbour = min_number
    
    # check right neighbour
    if((mynumber + 1) not in mylist):
        right_neighbour = mynumber + 1
    else:
        max_number = mynumber
        while max_number in mylist:
            max_number = max_number + 1
        right_neighbour = max_number
    
    return left_neighbour, right_neighbour

In [4]:
def lstm_model():
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True, 
            kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(8, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X_train.shape[1])(L2)
    L4 = LSTM(8, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X_train.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='mae')
    return model

In [5]:
def train_anomaly_removal(df_train):
    
    # extract indexes for anomalies
    indexes = list(df_train[df_train.is_anomaly == 1].index)

    # creating a new df that replaces the anomalous samples with interpolation value
    df = pd.DataFrame(columns = df_train.columns)
    for i in range(0, len(df_train)):

        #print(i)

        # add all non-anomalies
        if(df_train.is_anomaly[i] == 0):
            df = df.append({'timestamp' : df_train.timestamp[i], 'value' : df_train.value[i], 'is_anomaly' : df_train.is_anomaly[i]},
            ignore_index = True)

        if((df_train.is_anomaly[i]==1) & (i != (len(df_train)-1))):
            if(df_train.is_anomaly[i+1]!=1):
                #print(i)
                value_interpolation = (df_train.value[interpolation_indexes(indexes, i)[0]]+df_train.value[interpolation_indexes(indexes, i)[1]])/2

                df = df.append({'timestamp' : df_train.timestamp[i], 'value': value_interpolation, 'is_anomaly' : 0.0},
            ignore_index = True)
    return df

In [6]:
def threshold_computing_max(X_train):
    X_train_pred = model.predict(X_train, verbose=0)
    train_mae_loss_avg = np.mean(np.abs(X_train_pred - X_train), axis=1)
    max_threshold = np.max(train_mae_loss_avg)
    return max_threshold

In [7]:
def reconstruction_loss_predictions(X_test):
    X_test_pred = model.predict(X_test, verbose=0)
    mae_loss = np.mean(np.abs(X_test_pred-X_test), axis=1)
    return mae_loss, X_test_pred

In [8]:
def predicted_labels(mae_loss, threshold):
    predicted_test_label = []
    for i in range(0, len(test_mae_loss)):
        if(test_mae_loss[i][0]>(threshold)):
            predicted_test_label.append(1)
        else:
            predicted_test_label.append(0)
    return predicted_test_label

In [9]:
path_files_yahoo = '../datasets/Yahoo_A1Benchmark'

## Extract all file names corresponding to time series

In [10]:
ts_names = []
for i in os.listdir(path_files_yahoo):
    ts_names.append(str(i.split('.csv')[0]))

In [11]:
len(ts_names)

67

In [12]:
df_final_results_details = pd.DataFrame(columns = ['TS_name', 'lstmae_reconstruction_loss'])
df_final_results = pd.DataFrame(columns = ['TS_name', 'Labels_True', 'Labels_Pred', 'Test_Size', 'Model'])

scaler = MinMaxScaler()

window = 168

for ts_name in tqdm(ts_names):
    
    label_pred_complete = []
    losses_complete = []
    
    print(ts_name)
    # path to train/test
    filename_yahoo = path_files_yahoo+ts_name+".csv"
    
    # read ts
    df_yahoo = pd.read_csv(filename_yahoo)
    
    
    # split into train and test
    init_train = df_yahoo[0:math.floor(len(df_yahoo)/2)]
    # print(len(init_train))
    init_test = df_yahoo[math.floor(len(df_yahoo)/2):]
    # print(len(init_test))
    
    
    for i in tqdm(range(0, (math.floor(len(init_test)/window)+1))):
        
        # adjust training over time
        df_yahoo_train = pd.concat([init_train, init_test[0:window*i]], ignore_index=True)
        print(len(df_yahoo_train))
        print(df_yahoo_train.iloc[len(df_yahoo_train)-1])
        
        # adjust testing over time
        
        if(i == (round(len(init_test)/window))):
            df_yahoo_test = init_test[(i)*window:]
        else:
            df_yahoo_test = init_test[(i*window):((i+1)*window)]
    
    
        # remove anomalies from train to prepare LSTM
        # all anomalies are replaced by the interpolation of their closest non-anomalous neighbours
        df_train_yahoo = train_anomaly_removal(df_yahoo_train)


        # final training dataset + labels
        label_train = df_train_yahoo.is_anomaly
        train_yahoo = df_train_yahoo.value
        


        # final testing dataset + labels
        label_test = df_yahoo_test.is_anomaly
        test_yahoo = df_yahoo_test.value
        

        # Data preprocessing - Scaling
        # the scaler is fit on the training data and applied on the testing data
        train_yahoo_scale = scaler.fit_transform(np.array(train_yahoo).reshape(-1, 1))
        test_yahoo_scale = scaler.transform(np.array(test_yahoo).reshape(-1,1))

        # Shape Train Data for LSTM
        X_train = train_yahoo_scale.reshape(train_yahoo_scale.shape[0], 1, 1)

        # Train LSTM
        no_epochs = 50
        batch_size = 128
        model = lstm_model()
        encdec = model.fit(X_train, X_train, epochs=no_epochs, batch_size=batch_size,
                            validation_split=0.25).history

        # Threshold computing
        threshold = threshold_computing_max(X_train)

        # Shape Test Data for LSTM
        X_test = test_yahoo_scale.reshape(test_yahoo_scale.shape[0], 1, 1)

        test_mae_loss, X_test_pred = reconstruction_loss_predictions(X_test)

        # Extracting Predicted Labels
        y_label_pred = predicted_labels(test_mae_loss, threshold)

        label_pred_complete.append(y_label_pred)
        losses_complete.append(threshold)
    
    all_predicted_labels = []
    for i in range(0, len(label_pred_complete)):
        for j in range(0, len(label_pred_complete[i])):
            all_predicted_labels.append(label_pred_complete[i][j])
    
    
    
    
    # Save Results
    # Save reconstruction Error for each Dataset
    df_results_details = pd.DataFrame()
    df_results_details['TS_name'] = [ts_name]
    df_results_details['lstmae_reconstruction_loss'] = [losses_complete]
    df_results_details['retraining_technique'] = 'full_history'
    df_results_details['retraining_window'] = window
    df_final_results_details = df_final_results_details.append(df_results_details)


    # Save Predicted Labels
    df_results = pd.DataFrame()


    df_results['TS_name'] = [ts_name]
    df_results['retraining_technique'] = 'full_history'
    df_results['retraining_window'] = window
    df_results['Labels_True'] = [list(init_test.is_anomaly)]
    df_results['Labels_Pred'] = [all_predicted_labels]
    df_results['Test_Size'] = len(list(init_test.is_anomaly))
    df_results['Model'] = 'LSTM_AE'
    df_final_results = df_final_results.append(df_results)
    

  0%|          | 0/5 [00:00<?, ?it/s]

real_59
711
timestamp     711.0000
value           0.5455
is_anomaly      0.0000
Name: 710, dtype: float64
Epoch 1/50
5/5 [==============================] - 3s 125ms/step - loss: 0.6677 - val_loss: 0.6157
Epoch 2/50
5/5 [==============================] - 0s 7ms/step - loss: 0.6607 - val_loss: 0.6093
Epoch 3/50
5/5 [==============================] - 0s 7ms/step - loss: 0.6538 - val_loss: 0.6027
Epoch 4/50
5/5 [==============================] - 0s 7ms/step - loss: 0.6465 - val_loss: 0.5961
Epoch 5/50
5/5 [==============================] - 0s 7ms/step - loss: 0.6392 - val_loss: 0.5892
Epoch 6/50
5/5 [==============================] - 0s 8ms/step - loss: 0.6315 - val_loss: 0.5821
Epoch 7/50
5/5 [==============================] - 0s 7ms/step - loss: 0.6239 - val_loss: 0.5749
Epoch 8/50
5/5 [==============================] - 0s 8ms/step - loss: 0.6157 - val_loss: 0.5674
Epoch 9/50
5/5 [==============================] - 0s 8ms/step - loss: 0.6075 - val_loss: 0.5596
Epoch 10/50
5/5 [==========


 20%|██        | 1/5 [00:05<00:23,  5.79s/it]

879
timestamp     879.000000
value           0.524822
is_anomaly      0.000000
Name: 878, dtype: float64
Epoch 1/50
6/6 [==============================] - 3s 93ms/step - loss: 0.6604 - val_loss: 0.6142
Epoch 2/50
6/6 [==============================] - 0s 7ms/step - loss: 0.6544 - val_loss: 0.6083
Epoch 3/50
6/6 [==============================] - 0s 7ms/step - loss: 0.6485 - val_loss: 0.6022
Epoch 4/50
6/6 [==============================] - 0s 7ms/step - loss: 0.6423 - val_loss: 0.5962
Epoch 5/50
6/6 [==============================] - 0s 7ms/step - loss: 0.6362 - val_loss: 0.5900
Epoch 6/50
6/6 [==============================] - 0s 7ms/step - loss: 0.6299 - val_loss: 0.5836
Epoch 7/50
6/6 [==============================] - 0s 7ms/step - loss: 0.6234 - val_loss: 0.5772
Epoch 8/50
6/6 [==============================] - 0s 7ms/step - loss: 0.6169 - val_loss: 0.5706
Epoch 9/50
6/6 [==============================] - 0s 7ms/step - loss: 0.6103 - val_loss: 0.5638
Epoch 10/50
6/6 [=============


 40%|████      | 2/5 [00:12<00:18,  6.10s/it]

...


 33%|███▎      | 1/3 [00:04<00:09,  4.59s/it]

538
timestamp     538.000000
value          14.497164
is_anomaly      0.000000
Name: 537, dtype: float64
Epoch 1/50
4/4 [==============================] - 3s 184ms/step - loss: 0.4916 - val_loss: 0.5062
Epoch 2/50
4/4 [==============================] - 0s 8ms/step - loss: 0.4862 - val_loss: 0.5005
Epoch 3/50
4/4 [==============================] - 0s 7ms/step - loss: 0.4804 - val_loss: 0.4947
Epoch 4/50
4/4 [==============================] - 0s 7ms/step - loss: 0.4745 - val_loss: 0.4888
Epoch 5/50
4/4 [==============================] - 0s 7ms/step - loss: 0.4685 - val_loss: 0.4828
Epoch 6/50
4/4 [==============================] - 0s 7ms/step - loss: 0.4624 - val_loss: 0.4765
Epoch 7/50
4/4 [==============================] - 0s 7ms/step - loss: 0.4562 - val_loss: 0.4702
Epoch 8/50
4/4 [==============================] - 0s 7ms/step - loss: 0.4500 - val_loss: 0.4638
Epoch 9/50
4/4 [==============================] - 0s 7ms/step - loss: 0.4436 - val_loss: 0.4572
Epoch 10/50
4/4 [============


 67%|██████▋   | 2/3 [00:09<00:04,  4.67s/it]

706
timestamp     706.000000
value          14.390467
is_anomaly      0.000000
Name: 705, dtype: float64
Epoch 1/50
5/5 [==============================] - 2s 85ms/step - loss: 0.4923 - val_loss: 0.4810
Epoch 2/50
5/5 [==============================] - 0s 6ms/step - loss: 0.4866 - val_loss: 0.4753
Epoch 3/50
5/5 [==============================] - 0s 6ms/step - loss: 0.4809 - val_loss: 0.4695
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 0.4751 - val_loss: 0.4636
Epoch 5/50
5/5 [==============================] - 0s 6ms/step - loss: 0.4693 - val_loss: 0.4577
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 0.4634 - val_loss: 0.4519
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 0.4575 - val_loss: 0.4462
Epoch 8/50
5/5 [==============================] - 0s 6ms/step - loss: 0.4515 - val_loss: 0.4405
Epoch 9/50
5/5 [==============================] - 0s 6ms/step - loss: 0.4456 - val_loss: 0.4348
Epoch 10/50
5/5 [=============


100%|██████████| 67/67 [32:30<00:00, 29.12s/it]


In [15]:
df_final_results = df_final_results.set_index([pd.Index(np.arange(len(df_final_results))), 'TS_name'])
df_final_results_details = df_final_results_details.set_index([pd.Index(np.arange(len(df_final_results_details))), 'TS_name'])


In [19]:
df_final_results.to_csv('./results/df_results_lstmae_yahoo_fh_168.csv')

In [20]:
df_final_results_details.to_csv('./results/df_results_details_lstmae_yahoo_fh_168.csv')